In [1]:
import sqlite3
import pandas as pd
import ast
import numpy as np

In [2]:
con = sqlite3.connect('cademycode.db')
cur = con.cursor()

In [3]:
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
print(table_list)

[('cademycode_students',), ('cademycode_courses',), ('cademycode_student_jobs',)]


In [4]:
students = pd.read_sql_query("SELECT * FROM cademycode_students", con)
career_paths = pd.read_sql_query("SELECT * FROM cademycode_courses", con)
student_jobs = pd.read_sql_query("SELECT * FROM cademycode_student_jobs", con)

In [10]:
print('students: ', len(students))
print('career_paths: ', len(career_paths))
print('student_jobs: ', len(student_jobs))

students:  5000
career_paths:  10
student_jobs:  13
